In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image


tfk = tf.keras
tfkl = tf.keras.layers

# Random seed for reproducibility
seed = 6


classes = 8


random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

dataset_dir = 'data1'
training_dir = os.path.join(dataset_dir, 'training')
validation_dir = os.path.join(dataset_dir, 'validation')
test_dir = os.path.join(dataset_dir, 'test')

# Images are divided into folders, one for each class.
# If the images are organized in such a way, we can exploit the
# ImageDataGenerator to read them from disk.
from keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator for training and validation sets
valid_data_gen = ImageDataGenerator(rescale=1/255.)
test_data_gen = ImageDataGenerator(rescale=1/255.)

# Create an instance of ImageDataGenerator with Data Augmentation
aug_train_data_gen = ImageDataGenerator(rotation_range=40,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='reflect',
                                        brightness_range=[0.5,1.5],
                                        rescale=1/255.) # rescale value is multiplied to the image

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=training_dir,
                                                       target_size=(96,96),
                                                       color_mode='rgb',
                                                       classes=None, # can be set to labels
                                                       class_mode='categorical',
                                                       batch_size=16,
                                                       shuffle=True,
                                                       seed=seed)

valid_gen = valid_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=16,
                                               shuffle=False,
                                               seed=seed)
test_gen = test_data_gen.flow_from_directory(directory=test_dir,
                                             target_size=(96, 96),
                                             color_mode='rgb',
                                             classes=None, # can be set to labels
                                             class_mode='categorical',
                                             batch_size=16,
                                             shuffle=False,
                                             seed=seed)



input_shape = (96, 96, 3)
epochs = 200

def build_model(input_shape):
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    filters = [32, 64, 128, 256, 512]
    kernel_size = [3, 3, 3, 3, 3]
    nof_convolutional_layers = 5

    lrelu = lambda x: tf.keras.activations.relu(x, alpha=0.01)

    #x = tfkl.UpSampling2D((2, 2))(input_layer)
    x = tfkl.Conv2D(128, 3, strides=2, padding="same")(input_layer)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Activation('relu')(x)

    for i in range(nof_convolutional_layers):
        x = tfkl.Conv2D( #no more separable
            filters=filters[i],
            kernel_size=kernel_size[i],
            padding = 'same',
            activation = 'relu',
            kernel_initializer = tfk.initializers.HeUniform(seed)
        )(x)
        x = tfkl.BatchNormalization()(x)
        if i != nof_convolutional_layers - 1:
            x = tfkl.MaxPooling2D()(x)
        else:
            x = tfkl.GlobalAveragePooling2D()(x)

    x = tfkl.Flatten(name='Flatten')(x)
    x = tfkl.Dropout(0.3, seed=seed)(x)
    x = tfkl.Dense(units=512, name='Classifier', kernel_initializer=tfk.initializers.HeUniform(seed), activation='relu')(x)
    x = tfkl.Dropout(0.3, seed=seed)(x)
    output_layer = tfkl.Dense(units=8, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='output_layer')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

# Utility function to create folders and callbacks for training
from datetime import datetime


def create_folders_and_callbacks(model_name):
    exps_dir = os.path.join('data_augmentation_experiments')
    if not os.path.exists(exps_dir):
        os.makedirs(exps_dir)

    now = datetime.now().strftime('%b%d_%H-%M-%S')

    exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)

    callbacks = []

    # Model checkpoint
    # ----------------
    ckpt_dir = os.path.join(exp_dir, 'ckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'),
                                                       save_weights_only=True,  # True to save only weights
                                                       save_best_only=False)  # True to save only the best epoch
    callbacks.append(ckpt_callback)

    # Visualize Learning on Tensorboard
    # ---------------------------------
    tb_dir = os.path.join(exp_dir, 'tb_logs')
    if not os.path.exists(tb_dir):
        os.makedirs(tb_dir)

    # By default shows losses and metrics for both training and validation
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                                 profile_batch=0,
                                                 histogram_freq=1)  # if > 0 (epochs) shows weights histograms
    callbacks.append(tb_callback)

    # Early Stopping
    # --------------
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
    callbacks.append(es_callback)

    return callbacks

# Build model (for data augmentation training)
model = build_model(input_shape)

#Training
# Create folders and callbacks and fit
aug_callbacks = create_folders_and_callbacks(model_name='CNN_Aug')

# Train the model
history = model.fit(
    x = aug_train_gen,
    epochs = epochs,
    validation_data = valid_gen,
    callbacks = aug_callbacks,
).history

# Save best epoch model
model.save("data_augmentation_experiments/TodaysBestSkip")

# Evaluate on test
# Trained with data augmentation
model_aug = tfk.models.load_model("data_augmentation_experiments/TodaysBestSkip")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with data augmentation")
print(model_aug_test_metrics)

Found 2609 images belonging to 8 classes.
Found 560 images belonging to 8 classes.
Found 560 images belonging to 8 classes.
Metal device set to: Apple M1 Pro
Epoch 1/200
164/164 [==============================] - 10s 54ms/step - loss: 2.1690 - accuracy: 0.2944 - val_loss: 6.2955 - val_accuracy: 0.1446
Epoch 2/200
164/164 [==============================] - 5s 30ms/step - loss: 1.8356 - accuracy: 0.3377 - val_loss: 6.0135 - val_accuracy: 0.1732
Epoch 3/200
164/164 [==============================] - 5s 28ms/step - loss: 1.6911 - accuracy: 0.3706 - val_loss: 3.3697 - val_accuracy: 0.2000
Epoch 4/200
164/164 [==============================] - 5s 28ms/step - loss: 1.5711 - accuracy: 0.4017 - val_loss: 1.4200 - val_accuracy: 0.4839
Epoch 5/200
164/164 [==============================] - 5s 28ms/step - loss: 1.4657 - accuracy: 0.4603 - val_loss: 3.0781 - val_accuracy: 0.2589
Epoch 6/200
164/164 [==============================] - 5s 28ms/step - loss: 1.4161 - accuracy: 0.4753 - val_loss: 1.8491 

164/164 [==============================] - 5s 28ms/step - loss: 0.5558 - accuracy: 0.7957 - val_loss: 0.6177 - val_accuracy: 0.7750
Epoch 57/200
164/164 [==============================] - 4s 27ms/step - loss: 0.5334 - accuracy: 0.8038 - val_loss: 1.0230 - val_accuracy: 0.6571


35/35 [==============================] - 1s 11ms/step - loss: 0.5592 - accuracy: 0.8143

Test metrics with data augmentation
{'loss': 0.5591956377029419, 'accuracy': 0.8142856955528259}
